In [ ]:
from non_dim_glider import glider
import casadi as ca
import numpy as np
import matplotlib.pyplot as plt
import pickle

In [ ]:
# N_list = [50, 100, 200, 400]
N = 200
solve_second_optim = False
using_opt_guess = False
use_upsampled_prior = True
energy_optimal = False

x_opt, u_opt, opt_guess, sol = glider(
    N=N,
    use_upsampled_prior=use_upsampled_prior,
    y_f=-50,
    x_f=100,
    solve_2nd_optim=solve_second_optim,
    using_opt_guess=using_opt_guess,
    energy_optimal=energy_optimal,
)

In [ ]:
tgrid = np.linspace(0, 1, N + 1)
final_x = x_opt[3][-1]
print(f"The final x is {final_x}")

# plot results
fig, ax = plt.subplots(nrows=2, ncols=2, figsize=(12, 8))
ax[0, 0].plot(x_opt[3], x_opt[4])
ax[0, 0].set_title("Trajectory")
ax[0, 1].plot(tgrid, x_opt[-1])
ax[0, 1].set_title("Beta")
ax[1, 0].plot(tgrid, x_opt[0])
ax[1, 0].plot(tgrid, x_opt[1])
ax[1, 0].legend(("u", "v"))
ax[1, 1].plot(tgrid, np.append(np.nan, u_opt[0]))
ax[1, 1].set_title("Control")
plt.show()

In [ ]:
np.save("x_opt", x_opt)
np.save("u_opt", u_opt)

# Open a file and use dump()
# if solve_second_optim:
#     pass
# else:
#     with open("opt_guess_with_beta_dot.pkl", "wb") as file:
#         # A new file will be created
#         pickle.dump(opt_guess, file)

In [ ]:
with open("opt_guess_non_dim.pkl", "wb") as file:
    # A new file will be created
    pickle.dump(opt_guess, file)

In [ ]:
# def construct_initial_guess(
#     x_opt: np.ndarray, u_opt: np.ndarray, N: int, p0: float
# ) -> list:
#     """
#     Construct an initial guess using a given solution.
#     """
#     w0 = []
#     for i in range(N + 1):
#         state = list(x_opt[:, i])
#         if i == 0:
#             w0.append([p0])
#             w0.append(state)
#             u = [u_opt[0, i]]
#             w0.append(u)
#         elif i == N:
#             for j in range(4):
#                 w0.append(state)
#         elif i > 0 and i < N:
#             for j in range(4):
#                 w0.append(state)
#             u = [u_opt[0, i]]
#             w0.append(u)
#     return w0


# w0 = construct_initial_guess(x_opt, u_opt, N=N, p0=17)
# w0_arr = np.concatenate(w0)
# np.save("w0", w0_arr)


# # This code takes a solution from a simulation with N control steps and
# # upsamples it so it can be used as an intial guess for a simulation with 2*N
# # control steps
# def upsampled_initial_guess(w0: list) -> list:
#     """
#     Creates an initial guess for a problem with 2*N control intervals.

#     Starts with a guess for a solution to the problem with N intervals.

#     Parameters
#     ----------
#     w0 : list
#         Initial control parameters.

#     Returns
#     -------
#     w_new : list
#         Initial guess for problem with 2*N control intervals.
#     """
#     w_new = []
#     w_new.append(w0[0])
#     w_new.append(w0[1])
#     new_N = 2 * N
#     for i in range(N):
#         for k in range(2):
#             w_new.append([u_opt[0, i]])
#             for j in range(4):
#                 state = list(x_opt[:, i + 1])
#                 w_new.append(state)
#     return w_new


# w_new = upsampled_initial_guess(w0)
# w_new_arr = np.concatenate(w_new)
# np.save("double_w0", w_new_arr)

In [ ]:
sol

In [ ]:
def construct_initial_guess_upsampled(
    p0: float, x_opt: np.ndarray, u_opt: np.ndarray, N: int, new_N: int
) -> list:
    """
    Construct an initial guess using a given solution.

    The initial guess has the form (p0, x0, u0, ....) where the
    ... is the sequence of states and controls that follows the
    initial state/control. The only trick is that the state
    gets copied 4 times after each control. This method creates
    a new "upsampled" guess based on a good old solution. By upsampled
    I simply mean it will serve as a guess for a problem with a larger
    N. I do the upsampling by simply repeating the state and control
    from the end of the current solution.

    Parameters
    ----------
    p0 : float
        Initial guess for the parameter.
    x_opt: np.ndarray
        Array containing state along the optimal solution.
    u_opt: np.ndarray
        Arracy containing the control along the optimal trajectory.
    N: int
        Original number of control intervals.
    new_N: int
        New number of control intervals.
    """
    w0 = [[p0]]
    print(new_N)
    for i in range(new_N + 1):
        if i > N - 1:
            k = -1
        else:
            k = i
        state = list(x_opt[:, k])
        if i == 0:
            w0.append(state)
            u = [u_opt[0, i]]
            w0.append(u)
        elif i == new_N:
            for j in range(4):
                w0.append(state)
        elif i > 0 and i < new_N:
            for j in range(4):
                w0.append(state)
            u = [u_opt[0, k]]
            w0.append(u)
    return w0


upsamp = construct_initial_guess_upsampled(
    p0=24, x_opt=x_opt, u_opt=u_opt, N=N, new_N=200
)
upsamp = np.concatenate(upsamp)

In [ ]:
np.save("upsamp_w0", upsamp)

In [ ]:
# # generate the videos
# create_video = solve_second_optim
# if create_video:
#     !python trajectory_viz.py
#     if energy_optimal:
#         !mkdir energy_opt
#         !mv u_opt.npy x_opt.npy logged_info.png sample_oct.mp4 w0.npy double_w0.npy energy_opt
#     else:
#         !mkdir time_opt
#         !mv u_opt.npy x_opt.npy logged_info.png sample_oct.mp4 w0.npy double_w0.npy time_opt

In [ ]:
!python trajectory_viz.py -p "./"

In [ ]:
!open sample_oct.mp4